In [1]:
%load_ext autoreload
%autoreload 2

from source.read_HAR_dataset import read_har_dataset, har_dimensions, har_activities, har_activities_map, har_ind_IDS
from source.utils import  filter_dimensions
from source.tserie import TSerie
from source.utils import classify_dataset
from itertools import chain, combinations
from sklearn.ensemble import AdaBoostClassifier
from xgboost import XGBClassifier
from sklearn import svm
from source.utils import idsStd
from sklearn import metrics
import numpy as np
from sklearn import svm
import umap
from imblearn.under_sampling import RandomUnderSampler
from source.augmentation import  * 
# from cuml.datasets import make_blobs
# from cuml.neighbors import NearestNeighbors
# from cuml.manifold import UMAP
# from cuml.cluster import DBSCAN


sys.path.insert(0, '/home/texs/Documentos/Repositories/mts_viz')
from server.source.storage import MTSStorage

def powerset(iterable):
    "powerset([1,2,3]) --> () (1,) (2,) (3,) (1,2) (1,3) (2,3) (1,2,3)"
    s = list(iterable)  # allows duplicate elements
    return chain.from_iterable(combinations(s, r) for r in range(len(s)+1))

N_COMPONENTS=32
Z_SCORE_NORM = False
DATASET = 'HAR-UML20'
KFOLDS = 1
N_TESTS = 2
METRIC  = 'braycurtis'
RESULTS_PATH = 'outputs/augmentation/'
# AUGMENTATIONS = ['rotation', 'permutation', 'time_warp', 'magnitude_warp', 'scaling', 'jitter']
# AUGMENTATIONS = ['none']
AUGMENTATIONS = ['scaling']
ALL_AUGMENTATIONS = ['none', 'rotation', 'permutation', 'time_warp', 'magnitude_warp', 'scaling', 'jitter']
# ALL_AUGMENTATIONS = ['none', 'rotation', 'rotation', 'rotation', 'rotation', 'rotation', 'rotation']
# ALL_AUGMENTATIONS = ['none', 'rotation'] 
# AUGMENTATIONS = ['magnitude_warp']
REPEATS_PER_AUGMENTATION = 1
INCLUDE_ORIGINAL = True
# N_DIMS_NAMES = ['Acc', 'Gyro', 'Mag']
N_DIMS_NAMES = ['Acc', 'Gyro']
# N_DIMS_NAMES = ['Acc']
N_DIMENSIONS = [
    [
        'Accelerometer-X',	
        'Accelerometer-Y',	
        'Accelerometer-Z',
    ],
    [
        'Gyrometer-X',
        'Gyrometer-Y',
        'Gyrometer-Z',
    ],
    # [
    #     'Magnetometer-X',
    #     'Magnetometer-Y',
    #     'Magnetometer-Z'
    # ]
]



/home/texs/anaconda3/envs/contrastive/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
class FeatureExtractor:
    def __init__(self, epochs = 100, batch_size = 32, loss_metric = 'SimCLR', encoding_size = 8, mode = 'subsequences'):
        self.model = None
        self.device = None
        self.epochs = epoch
        self.batch_size = batch_size
        self.loss_metric = loss_metric
        self.encoding_size = encoding_size

    def fit_transform(self, X, y=None):
        self.nearNeigh.fit(X)
        knn_graph = self.nearNeigh.kneighbors_graph(X, mode="distance")
        embeddings =  self.reducer.fit_transform(X, y=y, knn_graph=knn_graph.tocsr(), convert_dtype=True)
        return embeddings
    
    def transform(self, X):
        knn_graph = self.nearNeigh.kneighbors_graph(X, mode="distance")
        embeddings =  self.reducer.transform(X, knn_graph=knn_graph.tocsr(), convert_dtype=True)
        return embeddings


def center(X_train, X_test, train_ind, test_ind, dimensions ):
    N_tr, T, D = X_train.shape
    N_te, T, D = X_test.shape
    X_train_sh = np.zeros(X_train.shape)
    X_test_sh = np.zeros(X_test.shape)
    for i in range(N_tr):
        
        for k in range(D):
            if k in dimensions:
                mag = np.mean(X_train[i, :, k], axis = 0)
                indice = np.where(train_ind ==  I_train[i])[0][0]
                std = ind_std_train[indice][k] * 6
                X_train_sh[i, :, k] = (X_train[i, :, k] - mag)
                # X_train_sh[i, :, k] = np.concatenate([[0], fft(X_train[i, :, k])[1:]])
            else:
                X_train_sh[i, :, k] = X_train[i, :, k]

    for i in range(N_te):
        for k in range(D):
            if k in dimensions:
                mag = np.mean(X_test[i, :, k], axis = 0)
                indice = np.where(test_ind ==  I_test[i])[0][0]
                std = ind_std_test[indice][k] * 6
                X_test_sh[i, :, k] = (X_test[i, :, k] - mag)
                # X_test_sh[i, :, k] = fft(X_test[i, :, k])[0:]
            else:
                X_test_sh[i, :, k] = X_test[i, :, k]
    return X_train_sh, X_test_sh

def znorm(X_train, X_test, train_ind, test_ind, ind_std_train, ind_std_test, dimensions):
    N_tr, T, D = X_train.shape
    N_te, T, D = X_test.shape
    X_train_sh = np.zeros(X_train.shape)
    X_test_sh = np.zeros(X_test.shape)
    for i in range(N_tr):
        
        for k in range(D):
            if k in dimensions:
                mag = np.mean(X_train[i, :, k], axis = 0)
                indice = np.where(train_ind ==  I_train[i])[0][0]
                std = ind_std_train[indice][k] * 6
                X_train_sh[i, :, k] = (X_train[i, :, k] - mag) / std
            else:
                X_train_sh[i, :, k] = X_train[i, :, k]

    for i in range(N_te):
        for k in range(D):
            if k in dimensions:
                mag = np.mean(X_test[i, :, k], axis = 0)
                indice = np.where(test_ind ==  I_test[i])[0][0]
                std = ind_std_test[indice][k] * 6
                X_test_sh[i, :, k] = (X_test[i, :, k] - mag) / std
            else:
                X_test_sh[i, :, k] = X_test[i, :, k]
    return X_train_sh, X_test_sh

def augmentData(X, y, augmentation, repeat = 3):
    X_out = []
    y_out = []
    for i in range(repeat):
        if augmentation == 'rotation':
            augmented = rotation(X, angle_range=[-np.pi/8, np.pi/8])
            # augmented = rotation(X, angle_range=[-np.pi/64, np.pi/64])
        elif augmentation == 'permutation':
            augmented = permutation(X)
        elif augmentation == 'time_warp':
            augmented = time_warp(X, sigma=0.03)
        elif augmentation == 'magnitude_warp':
            augmented = magnitude_warp(X, sigma=0.04, knot=4)
        elif augmentation == 'scaling':
            augmented = scaling(X, sigma=0.05)
        elif augmentation == 'jitter':
            augmented = jitter(X, sigma=0.01)
        else:
            augmented = X
        if len(X_out) == 0:
            X_out = augmented
            y_out = y
        else:
            X_out = np.concatenate((X_out, augmented), axis=0)
            y_out = np.concatenate((y_out, y), axis=0)
    return X_out, y_out

def augment(X, y, augmentations, repeats_per_augmentation=1, include_original=False):
    X_aug = []
    y_aug = []
    if include_original:
        X_aug = X
        y_aug = y
    for augmentation in augmentations:
        curr_X_aug, curr_y_aug = augmentData(X, y, augmentation, repeat=repeats_per_augmentation)
        if len(X_aug) == 0:
            X_aug = curr_X_aug
            y_aug = curr_y_aug
        else:
            X_aug = np.concatenate((X_aug, curr_X_aug), axis=0)
            y_aug = np.concatenate((y_aug, curr_y_aug), axis=0)
    return X_aug, y_aug

def minoritySampling(X, y):
    rus = RandomUnderSampler(sampling_strategy='not minority', random_state=1)
    N, T, D = X.shape
    X_temp = X.reshape([N, T * D])
    X_temp, y = rus.fit_resample(X_temp, y)
    X = X_temp.reshape([X_temp.shape[0], T, D])
    return X, y

In [3]:
from source.torch_utils import getContrastiveFeatures


firstTimeSave = True
storage = MTSStorage('har_augmentations')
storage.delete()
storage.load()


components_map = {}

for k in range(KFOLDS):
    # ------------------------ Reading the dataset ------------------------
    if DATASET == 'HAR-UML20':
        all_ids = har_ind_IDS
        test_ids = all_ids[k: k + N_TESTS]
        train_ids = all_ids[:k] + all_ids[k + N_TESTS:]        
        
        data = read_har_dataset('./datasets/HAR-UML20/', train_ids=train_ids, test_ids=test_ids, val_ids=[], cache=True)
        ids_train, X_train, y_train, I_train, train_kcal_MET = data['train']
        # ids_val, X_val, y_val, I_val, val_kcal_MET = data['val']
        ids_test, X_test, y_test, I_test, test_kcal_MET = data['test']
        
        har_activities_map = {
            0: "Sedentary",
            1: "Walking",
            2: "Running",
            3: "Downstairs",
            4: "Upstairs"
        }

        all_dimensions = har_dimensions
        activities_map = har_activities_map
        
        y_train[y_train==0] = 0
        y_train[y_train==1] = 0
        y_train[y_train==2] = 0
        y_test[y_test==0] = 0
        y_test[y_test==1] = 0
        y_test[y_test==2] = 0

        for i in range(3, len(har_activities)):
            y_train[y_train==i] = i - 2
            y_test[y_test==i] = i - 2
        
        ind_std_train = idsStd(train_ids , X_train, I_train)
        ind_std_test = idsStd(test_ids, X_test, I_test)
        
        # unique, counts = np.unique(y_train, return_counts=True)
        # unique, counts = np.unique(y_test, return_counts=True)
        X_train, y_train = minoritySampling(X_train, y_train)
        X_test, y_test = minoritySampling(X_test, y_test)
        # X_train_o = X_train.copy()
        # y_train_o = y_train.copy()
        # X_test_o = X_test.copy()
        # y_test_o = y_test.copy()
        
    # ---------------------------------------------------------------------

    
    # --------------------------------AugmentData ---------------------------------------------------
    if Z_SCORE_NORM:
        # X_train, X_test = center(X_train, X_test, train_ids, test_ids, dimensions = [0, 1, 2])
        X_train, X_test = znorm(X_train, X_test, train_ids, test_ids, ind_std_train, ind_std_test, dimensions = [0, 1, 2, 3, 4, 5])
        # X_test_o = X_test.copy()
    
    # -----------------------------------------------------------------------------------------------

    train_embeddings = []
    train_ys = []
    test_embeddings = []
    dimension_model = []
    
    # idx_trans = np.random.choice(np.arange(len(y_train_o)), len(y_train_o) // (len(ALL_AUGMENTATIONS) - 1), replace=False)
    # Y_
    
    augment_idx = []
    all_y = []
    
    cont = 0
    for aug in ALL_AUGMENTATIONS:
        if aug == 'none':
            idx_trans = np.random.choice(np.arange(len(y_train)), len(y_train), replace=False)
        else:
            n = len(y_train)
            nsize = len(y_train) // (len(ALL_AUGMENTATIONS) - 1)
            begin = cont * nsize
            end = (cont + 1) * nsize
            idx_trans = np.arange(len(y_train))[begin: end]
            # idx_trans = np.random.choice(np.arange(len(y_train_o)), , replace=False)
            cont = cont + 1
        # idx_trans = np.random.choice(np.arange(len(y_train_o)), len(y_train_o) // (len(ALL_AUGMENTATIONS)), replace=False)
        augment_idx.append(idx_trans)
        all_y.append(y_train[idx_trans])
        
    for t in range(len(N_DIMENSIONS)):
        dimensions = N_DIMENSIONS[t]    
        X_train_f = filter_dimensions(X_train, all_dimensions, dimensions)
        X_test_f = filter_dimensions(X_test, all_dimensions, dimensions)
        # X_test_f_o = filter_dimensions(X_test_o, all_dimensions, dimensions)
        # X_train_f_o = filter_dimensions(X_train_o, all_dimensions, dimensions)
        
        mts_train = TSerie(X = X_train_f, y = y_train, I = I_train, dimensions = dimensions, classLabels=activities_map)
        mts_test = TSerie(X = X_test_f, y = y_test, I = I_test, dimensions = dimensions, classLabels=activities_map)
        
        minl, maxl = mts_train.minMaxNormalization()
        mts_test.minMaxNormalization(minl=minl, maxl=maxl)
        
        # mts_train.folding_features_v2()
        # mts_test.folding_features_v2()
        
        n_neighbors = 15
        
        # reducer = umap.UMAP(n_components=N_COMPONENTS, metric=METRIC, n_neighbors=n_neighbors)
        # embeddings_train = reducer.fit_transform(mts_train.features, y=mts_train.y)
        # embeddings_test = reducer.transform(mts_test.features)
        embeddings_train, model, device =  getContrastiveFeatures(mts_train.X.transpose([0, 2, 1]), mts_train.y, epochs =20, loss_metric='SupConLoss',)
        
        # embeddings_train, embeddings_test = getContrastiveFeatures(mts_train.X.transpose([0, 2, 1]), mts_train.y, epochs =20, loss_metric='SupConLoss', X_test=mts_test.X.transpose([0, 2, 1]))
        
        # Create test augmentations 
        # TODO sample
        test_tranformations = [
            augment(mts_test.X, mts_test.y, repeats_per_augmentation = 1, augmentations = [ALL_AUGMENTATIONS[i]], include_original = False)
            for i in range(len(ALL_AUGMENTATIONS))
        ]
        
        train_tranformations = [
            augment(mts_train.X, mts_train.y, repeats_per_augmentation = 1, augmentations = [ALL_AUGMENTATIONS[i]], include_original = False)
            for i in range(len(ALL_AUGMENTATIONS))
        ]
        
        test_map = {}
        train_map = {}
        
        for i in range(len(ALL_AUGMENTATIONS)):
            tranformation_name = ALL_AUGMENTATIONS[i]
            # X_trans, y_trans =  augment(X_test_f_o.copy(), y_test, repeats_per_augmentation = 1, augmentations = [ALL_AUGMENTATIONS[i]], include_original = False)
            test_tranformations[i] = test_tranformations[i][0].copy()
            train_tranformations[i] = train_tranformations[i][0].copy()
            
            mts_test_t = TSerie(X = test_tranformations[i], dimensions = dimensions, classLabels=activities_map)
            # mts_test_t.folding_features_v2()
            
            # n_samples = None
            # if tranformation_name == 'none':
            #     idx = np.random.choice(np.arange(len(y_train_o)), len(y_train_o), replace=False)
            # else:
            #     idx_trans = np.random.choice(np.arange(len(y_train_o)), len(y_train_o) // (len(ALL_AUGMENTATIONS) - 1), replace=False)
            #     idx = idx_trans

            idx = augment_idx[i]
            
            mts_train_t = TSerie(X = train_tranformations[i][idx], y = y_train[idx], dimensions = dimensions, classLabels=activities_map)
            # mts_train_t.folding_features_v2()
            
            test_map[ALL_AUGMENTATIONS[i]] = model.encode(mts_test_t.X.transpose([0, 2, 1]), device)
            train_map[ALL_AUGMENTATIONS[i]] = model.encode(mts_train_t.X.transpose([0, 2, 1]), device)
            # train_map_y[ALL_AUGMENTATIONS[i]] = mts_train_t.y
            
            if firstTimeSave:
                # print(np.random.random([test_tranformations[i].shape[0], 2]).shape)
                storage.add_mts(
                    'test {}_{}'.format(ALL_AUGMENTATIONS[i], N_DIMS_NAMES[t]),
                    mts_test_t.X, 
                    dimensions,
                    coords={
                        'umap': np.random.random([test_tranformations[i].shape[0], 2])
                    }, 
                    # labels={
                    #     'activities': mts_test.y, 
                    #     # 'participants': mts_test.I
                    # }, 
                    # labelsNames={'activities': activities_map },
                    sampling = True,
                    n_samples = 400
                )
                storage.save()
            
        
        # dimension_model.append(reducer)
        train_embeddings.append(train_map)
        # train_ys.append(train_map_y)
        test_embeddings.append(test_map)
    firstTimeSave = False
    
    names_comb = []
    embeddings_comb = []
    for i, combo in enumerate(powerset(list(range(len(N_DIMS_NAMES)))), 1):
        indexes = list(combo)
        name = ''
        train_embedding = {}
        # train_y = {}
        test_embedding = {}
        if len(indexes) == 0:
            continue
        for ind in indexes:
            name = name + ' ' + N_DIMS_NAMES[ind]
            if len(test_embedding) == 0:
                for aug in ALL_AUGMENTATIONS:
                    test_embedding[aug] = test_embeddings[ind][aug]
            else:
                for aug in ALL_AUGMENTATIONS:
                    test_embedding[aug] = np.concatenate([test_embedding[aug], test_embeddings[ind][aug]], axis=1)        

            if len(train_embedding) == 0:
                for aug in ALL_AUGMENTATIONS:
                    train_embedding[aug] = train_embeddings[ind][aug]
                    # train_y[aug] = train_ys[ind][aug]
            else:
                for aug in ALL_AUGMENTATIONS:
                    train_embedding[aug] = np.concatenate([train_embedding[aug], train_embeddings[ind][aug]], axis=1)
                    # train_y[aug] = np.concatenate([train_y[aug], train_ys[ind][aug]], axis=0)
            
            # if len(test_embedding) == 0:
            # for aug in ALL_AUGMENTATIONS:
            #     if len(train_embedding) == 0:
            #         train_embedding = train_embeddings[ind][aug]
            #         # train_y = train_ys[ind][aug]
            #     else:
            #         train_embedding = np.concatenate([train_embedding, train_embeddings[ind][aug]], axis=1)
                    # train_y = np.concatenate([train_y, train_ys[ind][aug]], axis=0)
            # else:
            #     for aug in ALL_AUGMENTATIONS:
            #         train_embedding = np.concatenate([train_embedding, train_embeddings[ind][aug]], axis=1)
            #         train_y = np.concatenate([train_y, train_ys[ind][aug]], axis=1)
            #         test_embedding[aug] = np.concatenate([test_embedding[aug], test_embeddings[ind][aug]], axis=1)        
        # print(train_ys[ind]['none'])
        # ind = indexes[1]
        # print(train_ys[ind]['none'].shape)
        # print(train_ys[ind]['rotation'].shape)
        # train_y = np.concatenate([train_ys[ind]['none'], train_ys[ind]['rotation']], axis=0)
        
        names_comb.append(name)
        embeddings_comb.append((train_embedding, test_embedding))
    
    for j in range(len(names_comb)):
        name = names_comb[j]
        # clf = AdaBoostClassifier()
        clf = XGBClassifier()
        # clf = svm.SVC()
        
        train_feat = []
        train_y = []
        
        train_feat_map, test_feat_map = embeddings_comb[j]
        
        for a in range(len(ALL_AUGMENTATIONS)):
            aug = ALL_AUGMENTATIONS[a]
            if(len(train_feat)) == 0:
                train_feat = train_feat_map[aug]
                train_y = all_y[a]
            else:
                train_feat = np.concatenate([train_feat, train_feat_map[aug]], axis = 0) 
                train_y = np.concatenate([train_y, all_y[a]], axis = 0) 
            # if a == 5:
            #     break
        # train_y = 
        clf.fit(train_feat, train_y)
        
        pred_train = clf.predict(train_feat)
        f1_tr = metrics.f1_score(train_y, pred_train, average='weighted')
        
        f1_scores = [f1_tr]
        
        for aug in ALL_AUGMENTATIONS:
            test_feat = test_feat_map[aug]
            pred_test = clf.predict(test_feat)
            f1_te = metrics.f1_score(y_test, pred_test, average='weighted')
            f1_scores.append(f1_te)
        
        if name not in components_map:
            components_map[name] = [f1_scores]
        else:
            components_map[name] = components_map[name] + [f1_scores]


        

Train IDS: [2, 3, 4, 5, 6, 7, 8, 9]
Test IDS: [0, 1]
Val IDS: []
Loading dataset from cache...
Subsequence length: 180
Using contrastive metric!!!!!!!!!!!!
Epoch[1] Train loss    avg: 4.077798395292134


KeyboardInterrupt: 

In [ ]:
# train_feat.shape

# train_y.shape

In [ ]:
import csv

path = os.path.join(RESULTS_PATH, 'Umap_kfold_{}.csv'.format('_'.join(AUGMENTATIONS)))
with open(path, 'w', newline='') as csvfile:
    row = ['Sensors', 'f1 train', 'f1 test']
    spamwriter = csv.writer(csvfile, delimiter=',',
                            quoting=csv.QUOTE_MINIMAL)
    spamwriter.writerow(row)
    for name in names_comb:
        row = [name]
        f1_mean_tr = np.array([ f1[0] for f1 in components_map[name]]).mean()
        f1_stds_tr = np.array([ f1[0] for f1 in components_map[name]]).std()
        f1_mean_te = np.array([ f1[1] for f1 in components_map[name]]).mean()
        f1_stds_te = np.array([ f1[1] for f1 in components_map[name]]).std()
        f1_mean_te_rot = np.array([ f1[2] for f1 in components_map[name]]).mean()
        f1_stds_te_rot = np.array([ f1[2] for f1 in components_map[name]]).std()
        f1_mean_te_per = np.array([ f1[3] for f1 in components_map[name]]).mean()
        f1_stds_te_per = np.array([ f1[3] for f1 in components_map[name]]).std()
        f1_mean_te_tim = np.array([ f1[4] for f1 in components_map[name]]).mean()
        f1_stds_te_tim = np.array([ f1[4] for f1 in components_map[name]]).std()
        f1_mean_te_mag = np.array([ f1[5] for f1 in components_map[name]]).mean()
        f1_stds_te_mag = np.array([ f1[5] for f1 in components_map[name]]).std()
        f1_mean_te_sca = np.array([ f1[6] for f1 in components_map[name]]).mean()
        f1_stds_te_sca = np.array([ f1[6] for f1 in components_map[name]]).std()
        f1_mean_te_jit = np.array([ f1[7] for f1 in components_map[name]]).mean()
        f1_stds_te_jit = np.array([ f1[7] for f1 in components_map[name]]).std()
        
        row = [
            name, 
            '{:.3f} ({:.3f})'.format(f1_mean_tr, f1_stds_tr), 
            '{:.3f} ({:.3f})'.format(f1_mean_te, f1_stds_te), 
            '{:.3f} ({:.3f})'.format(f1_mean_te_rot, f1_stds_te_rot), 
            '{:.3f} ({:.3f})'.format(f1_mean_te_per, f1_stds_te_per), 
            '{:.3f} ({:.3f})'.format(f1_mean_te_tim, f1_stds_te_tim), 
            '{:.3f} ({:.3f})'.format(f1_mean_te_mag, f1_stds_te_mag), 
            '{:.3f} ({:.3f})'.format(f1_mean_te_sca, f1_stds_te_sca), 
            '{:.3f} ({:.3f})'.format(f1_mean_te_jit, f1_stds_te_jit), 
        ]
        print(row)
        spamwriter.writerow(row)
            

[' Acc', '1.000 (0.000)', '0.835 (0.000)', '0.479 (0.000)', '0.814 (0.000)', '0.819 (0.000)', '0.663 (0.000)', '0.676 (0.000)', '0.712 (0.000)']


In [ ]:
'Centered  0.966 (0.017)'

'Centered  0.966 (0.017)'

In [ ]:
'Rotation 0.999 (0.001)', '0.955 (0.027)'

('Rotation 0.999 (0.001)', '0.955 (0.027)')

In [ ]:
'Original Acc', '0.998 (0.001)', '0.925 (0.045)'

('Original Acc', '0.998 (0.001)', '0.925 (0.045)')